In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'pf-henry-405314' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

# Librerias

In [ ]:
import pandas as pd

import datetime

from google.cloud import storage
from google.cloud import bigquery
import pandas
import pytz

from google.colab import drive
from google.colab import auth

from pandas import json_normalize

from textblob import TextBlob


# Archivo de metadatas de sitios de Google maps

In [ ]:
# se extrae arcivos .json de drive para crear una unica tabla de metadatos de sitios

def clean_category_column(df):
    df['category'] = df['category'].astype(str)
    df['category'] = df['category'].str.replace('[', '', regex=True).str.replace(']', '', regex=True)

# Montar el drive
drive.mount('/content/drive')

# Acceder a la carpeta compartida
folder_path = "/content/drive/My Drive/Google Maps/metadata-sitios/"

# Crear un diccionario para almacenar los dataframes
dataframes = []

# Leer y limpiar cada archivo .json
for i in range(1, 12):  #rango según la cantidad de archivos .json
    filename = str(i) + '.json'
    file_path = folder_path + filename
    data = pd.read_json(file_path, lines=True)
    data.drop(columns=['description','price','state','relative_results','url','hours'], inplace=True)
    clean_category_column(data)
    dataframes.append(data)

# Concatenar los dataframes en uno solo
result1 = pd.concat(dataframes, ignore_index=True)

result2 = pd.json_normalize(result1['MISC']).set_index(result1['MISC'].index)

result3 = pd.concat([result1, result2], axis=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
result3.drop('MISC', axis=1, inplace=True)
result3 = result3[result3['category'].str.contains('restaurants|restaurante|restaurant|food|takeaway|Lunch|Deli', case=False, na=False)]
result3 = result3.drop(columns=['Recycling','Getting here','Activities','Lodging options'])
result3.fillna('No', inplace=True)

In [ ]:
# se dividen la tabla principal en 2, una de sitios y otra de atributos

sitios = result3[['name', 'address', 'gmap_id', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews']]

In [ ]:
# se dividen la tabla principal en 2, una de sitios y otra de atributos

atributos = result3[['gmap_id', 'Service options','Accessibility', 'Offerings', 'Amenities', 'Atmosphere', 'Popular for', 'Dining options', 'Crowd']]

In [ ]:
atributos = atributos.join(atributos['Service options'].apply(pd.Series).add_prefix('Service'))
atributos = atributos.join(atributos['Accessibility'].apply(pd.Series).add_prefix('Accessibility'))
atributos = atributos.join(atributos['Offerings'].apply(pd.Series).add_prefix('Offerings'))


atributos.drop(columns=['Service options','Accessibility','Offerings'], inplace=True)

atributos.drop(columns=['Service3', 'Service4', 'Service5', 'Service6', 'Service7', 'Service8','Accessibility1',
       'Accessibility2', 'Accessibility3', 'Accessibility4', 'Accessibility5','Offerings3', 'Offerings4',
       'Offerings5', 'Offerings6', 'Offerings7', 'Offerings8', 'Offerings9', 'Offerings10', 'Offerings11', 'Offerings12', 'Offerings13',
       'Offerings14', 'Offerings15', 'Offerings16', 'Offerings17', 'Offerings18'], inplace=True)

In [ ]:
atributos = atributos.join(atributos['Amenities'].apply(pd.Series).add_prefix('Amenities'))
atributos = atributos.join(atributos['Atmosphere'].apply(pd.Series).add_prefix('Atmosphere'))
atributos = atributos.join(atributos['Crowd'].apply(pd.Series).add_prefix('Crowd'))

atributos.drop(columns=['Amenities','Atmosphere','Crowd'], inplace=True)

atributos.drop(columns=['Amenities1', 'Amenities2', 'Amenities3','Amenities4', 'Amenities5', 'Atmosphere1', 'Atmosphere2', 'Atmosphere3', 'Atmosphere4', 'Atmosphere5','Crowd1', 'Crowd2', 'Crowd3', 'Crowd4'], inplace=True)
atributos.fillna('No', inplace=True)

In [ ]:
sitios.reset_index(drop=True, inplace=True)

In [ ]:
# se carga el la tabla sitios a bigquery

auth.authenticate_user()
print('Authenticated')

# Especificar el proyecto y el dataset de BigQuery
project_id = 'pf-henry-405314'

# Crear un cliente de BigQuery
client = bigquery.Client(project=project_id)

table_id = 'pf-henry-405314.Unificados.sitios'

# Cargar el DataFrame a BigQuery
job_config =  bigquery.LoadJobConfig( schema =  [
                bigquery.SchemaField('name',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('address',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('gmap_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('latitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('longitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('category',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('avg_rating',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('num_of_reviews',	bigquery.enums.SqlTypeNames.INTEGER),
                ],
                write_disposition="WRITE_TRUNCATE",)


job = client.load_table_from_dataframe(sitios, table_id, job_config=job_config)  # llamada a la API

job.result()  # Espera mientras se ejecuta el trabajo

table = client.get_table(table_id)  # llamada a la API

print("Cargadas {} filas y {} columnas a {}".format(table.num_rows, len(table.schema), table_id))

Authenticated
Cargadas 240940 filas y 8 columnas a pf-henry-405314.Unificados.sitios


In [ ]:
atributos.drop(columns=['Popular for', 'Dining options'], inplace=True)
atributos.reset_index(drop=True, inplace=True)

In [ ]:
# se carga el la tabla atributos a bigquery

auth.authenticate_user()
print('Authenticated')

# Especificar el proyecto y el dataset de BigQuery
project_id = 'pf-henry-405314'

# Crear un cliente de BigQuery
client = bigquery.Client(project=project_id)

table_id = 'Unificados.atributos'

# Cargar el DataFrame a BigQuery
job_config =  bigquery.LoadJobConfig( schema =  [
                bigquery.SchemaField('gmap_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Accessibility0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Amenities0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Atmosphere0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Crowd0',	bigquery.enums.SqlTypeNames.STRING),
                ],
                write_disposition="WRITE_TRUNCATE",)


job = client.load_table_from_dataframe(atributos, table_id, job_config=job_config)  # llamada a la API

job.result()  # Espera mientras se ejecuta el trabajo

table = client.get_table(table_id)  # llamada a la API

print("Cargadas {} filas y {} columnas a {}".format(table.num_rows, len(table.schema), table_id))



Authenticated
Cargadas 240940 filas y 11 columnas a Unificados.atributos


In [ ]:
df = sitios.merge(atributos, on='gmap_id')
df.drop_duplicates(inplace=True)

In [ ]:
# se carga el la tabla sitios-atributos a bigquery

auth.authenticate_user()
print('Authenticated')

# Especificar el proyecto y el dataset de BigQuery
project_id = 'pf-henry-405314'

# Crear un cliente de BigQuery
client = bigquery.Client(project=project_id)

table_id = 'Unificados.sitios-atributos'

# Cargar el DataFrame a BigQuery
job_config =  bigquery.LoadJobConfig( schema =  [
                bigquery.SchemaField('name',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('address',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('gmap_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('latitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('longitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('category',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('avg_rating',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('num_of_reviews',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('Service0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Accessibility0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Amenities0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Atmosphere0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Crowd0',	bigquery.enums.SqlTypeNames.STRING),
                ],
                write_disposition="WRITE_TRUNCATE",)


job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  # llamada a la API

job.result()  # Espera mientras se ejecuta el trabajo

table = client.get_table(table_id)  # llamada a la API

print("Cargadas {} filas y {} columnas a {}".format(table.num_rows, len(table.schema), table_id))



Authenticated
Cargadas 239674 filas y 18 columnas a Unificados.sitios-atributos
